### Data Module

In [15]:
from typing import Dict, List
from loguru import logger
import numpy as np
import pandas as pd

In [16]:
BASE_ENCODING = {'BTC':0, 'ETH':1}

In [17]:
def load_data(start_date, end_date, base, path):
    """_summary_

    Args:
        start_date (_type_): 起始日期
        end_date (_type_): 结束日期
        base (_type_): 标的
        path (_type_): 数据路径

    Returns:
        _type_: DataFrame
    """
    df = pd.read_parquet(path) 
    assert 'date' in df.columns
    df['date'] = pd.to_datetime(df['date'])
    df_clip = df[(df['date']>=start_date) & (df['date'] <= end_date)]
    df_base = df_clip[df['base'] == base]
    return df_base
    

In [18]:
df = load_data('20200101', '20210101', 'btc', '/Users/zehoo/Documents/Course/波动率预测专题课程/data/DC_Features_20190401_20221224.parquet')
df

/var/folders/rc/d92q64ds74d0jg428sqj64lc0000gn/T/ipykernel_70351/990086325.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_base = df_clip[df['base'] == base]


,date,base,quote_x,open,high,low,close,volume,tradeval,ntrade,...,vwap_ask,twap_ask,awap_ask,open_asksize,high_asksize,low_asksize,close_asksize,vwa_asksize,twa_asksize,awa_asksize
13200,2020-01-01 00:00:00,btc,usdt,7200.52,7206.29,7185.76,7195.23,521.978397,3.755764e+06,6659,...,7195.993717,7195.455386,7195.633225,0.163159,8.656877,0.000001,0.031492,0.200801,0.175632,0.200791
13202,2020-01-01 01:00:00,btc,usdt,7195.24,7196.24,7175.46,7177.02,510.733521,3.668075e+06,7595,...,7182.098572,NaN,7181.940635,0.103091,3.899774,0.000001,0.228223,0.441340,NaN,0.441330
13204,2020-01-01 02:00:00,btc,usdt,7176.47,7230.00,7175.71,7216.27,883.052603,6.365953e+06,9033,...,7207.230252,7189.937233,7204.044510,0.228223,98.902845,0.000001,0.157927,0.787799,0.514536,0.787450
13206,2020-01-01 03:00:00,btc,usdt,7215.52,7244.87,7211.41,7242.85,655.156809,4.736719e+06,7466,...,7228.278972,7220.228511,7227.600090,0.008019,21.545003,0.000001,0.300000,0.576947,0.303303,0.576893
13208,2020-01-01 04:00:00,btc,usdt,7242.66,7245.00,7220.00,7225.01,783.724867,5.667367e+06,8337,...,7229.445759,7240.064157,7231.003755,0.640447,19.813351,0.000001,0.399879,0.669936,0.386292,0.670081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30760,2020-12-31 20:00:00,btc,usdt,28872.24,29000.00,28742.41,28897.83,2293.821339,6.628923e+07,56789,...,28899.936410,28896.987530,28896.765130,0.703973,52.079136,0.000001,0.244611,1.172446,1.151635,1.172317
30762,2020-12-31 21:00:00,btc,usdt,28897.84,29139.65,28862.00,29126.70,1936.480299,5.610330e+07,42101,...,28959.282340,28939.844050,28945.560650,0.243751,84.659713,0.000001,0.004500,1.164264,1.140160,1.163713
30764,2020-12-31 22:00:00,btc,usdt,29126.71,29169.55,28900.79,28966.36,2524.473111,7.335146e+07,51297,...,29061.244830,29049.671710,29051.604420,2.425720,38.106930,0.000001,0.491386,1.169931,1.140611,1.169543
30766,2020-12-31 23:00:00,btc,usdt,28966.36,29143.73,28910.19,29100.84,1438.506326,4.180712e+07,36413,...,29074.951920,29071.304470,29070.739290,0.501386,14.906356,0.000001,0.334847,0.659741,0.680326,0.659645


In [19]:
class DataModule:
    def __init__(self, source, freq:str, base:str='BTC') -> None:
        self.freq = freq
        self.base = base
        self.source = source
    
    def get_data(self, starttime, endtime, columns):
        df = load_data(start_date=starttime, end_date=endtime, base=self.base.lower(), path=self.source)
        df = df.sort_values(by=['date','base'], ascending=True)
        df.set_index(['date', 'base'], drop=True, inplace=True)
        df = df[columns]
        return df

In [20]:
COL_NAMES = ['open', 'high', 'low', 'close', 'vwap', 'twap', 'awap', 'volume','tradeval', 'ntrade', 'ntrade_up', 'ntrade_down', 'ntrade_flat', 'close_ask', 'close_bid', 'close_asksize', 'close_bidsize', 'close_spread', 'nquote']
ROOT_DIR = '/Users/zehoo/Documents/Course/波动率预测专题课程/data/DC_Features_20190401_20221224.parquet'
dm_btc = DataModule(source=ROOT_DIR, freq='1h', base='BTC')
dm_btc.get_data(starttime='20200102', endtime='20210102', columns=COL_NAMES)

/var/folders/rc/d92q64ds74d0jg428sqj64lc0000gn/T/ipykernel_70351/990086325.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_base = df_clip[df['base'] == base]


,,open,high,low,close,vwap,twap,awap,volume,tradeval,ntrade,ntrade_up,ntrade_down,ntrade_flat,close_ask,close_bid,close_asksize,close_bidsize,close_spread,nquote
date,base,,,,,,,,,,,,,,,,,,,
2020-01-02 00:00:00,btc,7197.62,7204.17,7175.15,7200.85,7188.047204,7197.715114,7189.798125,811.250160,5.831304e+06,7632,2573,2788,2271,7201.00,7200.14,0.102140,0.131905,0.86,16656
2020-01-02 01:00:00,btc,7200.08,7212.50,7185.46,7211.02,7200.446714,7200.821054,7200.155280,390.692131,2.813158e+06,5460,2014,1806,1640,7211.04,7210.69,0.346698,0.074837,0.35,11705
2020-01-02 02:00:00,btc,7211.04,7211.16,7185.11,7190.99,7194.751355,7195.567041,7195.165802,402.660139,2.897040e+06,5929,1927,2202,1800,7191.59,7190.99,0.347642,0.194713,0.60,10129
2020-01-02 03:00:00,btc,7190.99,7191.00,7155.35,7169.02,7167.135404,7166.878558,7166.781234,1387.377886,9.943525e+06,14716,4300,5136,5280,7169.24,7169.02,0.348718,0.288266,0.22,17764
2020-01-02 04:00:00,btc,7169.04,7169.06,7120.37,7129.61,7144.629838,7152.647616,7146.643096,1557.964430,1.113108e+07,15206,3795,4955,6456,7129.75,7128.98,0.064822,0.350663,0.77,15371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-01 20:00:00,btc,29072.70,29125.32,28950.00,29029.04,29047.320170,29071.678360,29046.680270,1629.228721,4.732473e+07,49038,12292,12177,24569,29029.04,29029.03,0.102240,1.169189,0.01,25092
2021-01-01 21:00:00,btc,29029.04,29279.72,28880.37,29200.96,29148.037800,29035.778590,29154.246660,1838.549082,5.359010e+07,52949,13811,12809,26329,29202.39,29200.96,0.213180,0.223545,1.43,25320
2021-01-01 22:00:00,btc,29200.97,29289.82,29130.00,29160.39,29218.142250,29201.483640,29213.139000,1447.229641,4.228536e+07,34181,8390,8999,16792,29162.71,29162.70,0.089571,0.021620,0.01,21799


In [31]:
def smoothing_Z_score(df:pd.DataFrame, sz:int, tr:int):
    df_copy = df.copy()
    for column in df.columns:
        # 计算滚动窗口内的均值和标准差
        df_copy[column + '_mean'] = df_copy[column].rolling(window=sz, min_periods=1).mean()
        df_copy[column + '_std'] = df_copy[column].rolling(window=sz, min_periods=1).std()
        
        # 计算异常值的阈值范围
        df_copy[column + '_lower_bound'] = df_copy[column + '_mean'] - tr * df_copy[column + '_std']
        df_copy[column + '_upper_bound'] = df_copy[column + '_mean'] + tr * df_copy[column + '_std']
        
        # 标记异常值
        outliers = (df_copy[column] < df_copy[column + '_lower_bound']) | (df_copy[column] > df_copy[column + '_upper_bound'])
        
        # 裁剪异常值
        df_copy.loc[outliers, column] = np.clip(df_copy.loc[outliers, column], 
                                                df_copy[column + '_mean'] - tr * df_copy[column + '_std'], 
                                                df_copy[column + '_mean'] + tr * df_copy[column + '_std'])   
    # 删除中间结果列
    df_copy = df_copy.drop(columns=[col for col in df_copy.columns if col.endswith(('_mean', '_std', '_lower_bound', '_upper_bound') )])
    return df_copy

In [36]:
class DataPreprocessing:
    def __init__(self, datasets: Dict, freq:str) -> None:
        self.dataset_names = datasets.keys()
        self.datasets = datasets
        self.freq = freq
    
    def _outlier_check(self, data:pd.DataFrame, sz:int=5, tr:int=3):
        data = smoothing_Z_score(data, sz=sz, tr=tr)
        return data
    
    def _missing_check(self, data:pd.DataFrame, dataset_name):
        stime = lambda x: x.strftime("%Y-%m-%d %H:%M:%S")
        begin, end = stime(data.index[0][0]), stime(data.index[-1][0])
        expected_data_row_number = len(list(pd.date_range(begin, end, freq = self.freq)))
        missing_rows = expected_data_row_number - data.shape[0]
        if missing_rows > 0:
            logger.warning("Missing %d rows for %s" % (missing_rows, dataset_name))
    
    def _data_filter(self, dataset_name):
        data = self.datasets[dataset_name].copy()
        data = self._outlier_check(data)
        self._missing_check(data, dataset_name)
        data = data.reset_index(level='base')
        data['base'] = data['base'].apply(lambda x: BASE_ENCODING[x.upper()])
        data = data.asfreq(self.freq)
        return data
        
    def data_collation(self):
        DatasetCollections = []
        for db_name in self.dataset_names:
            dbb_data = self._data_filter(db_name)
            dbb_data = dbb_data.reset_index(drop=False)
            DatasetCollections.append(dbb_data)
        data = pd.concat(DatasetCollections)
        return data.sort_values(['date', 'base'])
        

In [37]:
freq = '1h'
dm = [DataModule(ROOT_DIR, freq, base) for base in ['BTC', 'ETH']]
datasets = dict(zip(['BTC', 'ETH'], [dm_i.get_features(starttime='20200102', endtime='20210102', columns=COL_NAMES) for dm_i in dm]))
dp = DataPreprocessing(datasets=datasets, freq=freq)
data = dp.data_collation()


/var/folders/rc/d92q64ds74d0jg428sqj64lc0000gn/T/ipykernel_70351/990086325.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_base = df_clip[df['base'] == base]
/var/folders/rc/d92q64ds74d0jg428sqj64lc0000gn/T/ipykernel_70351/990086325.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_base = df_clip[df['base'] == base]
/var/folders/rc/d92q64ds74d0jg428sqj64lc0000gn/T/ipykernel_70351/3983087579.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_copy[column + '_lower_bound'] = df_copy[column + '_mean'] - tr * df_copy[column + '_std']
/var/folders/rc/d92q64ds74d0jg428sqj64lc0000gn/T/ipykernel_70351/3983087579.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usuall

In [38]:
data

,date,base,open,high,low,close,vwap,twap,awap,volume,...,relative_tradeval,relative_ntrade,relative_high_ratio,relative_low_ratio,corr_high_rank_volume,corr_std_high_vol,price_diff_ratio,corr_high_low,super_high_low,rv_24
0,2020-01-02 00:00:00,0,7197.62,7204.17,7175.15,7200.85,7188.047204,7197.715114,7189.798125,811.250160,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.402224
0,2020-01-02 00:00:00,1,130.28,130.87,129.74,130.77,130.270294,130.271626,130.293196,3865.459910,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.405712
1,2020-01-02 01:00:00,0,7200.08,7212.50,7185.46,7211.02,7200.446714,7200.821054,7200.155280,390.692131,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.409003
1,2020-01-02 01:00:00,1,130.71,130.78,130.27,130.67,130.551206,130.758253,130.524085,3769.696330,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.405849
2,2020-01-02 02:00:00,0,7211.04,7211.16,7185.11,7190.99,7194.751355,7195.567041,7195.165802,402.660139,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.426810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8782,2021-01-01 22:00:00,1,729.48,730.61,726.69,728.23,728.774104,729.453325,728.807794,14582.153160,...,0.844815,0.898394,0.963057,1.009324,-0.006285,-73.114013,0.839299,0.939944,0.833681,NaN
8783,2021-01-01 23:00:00,0,29163.17,29326.74,29104.57,29262.32,29255.752750,29164.501190,29253.241230,1127.035897,...,0.904756,0.928417,0.990768,1.071804,-0.194326,-664.458246,0.881032,0.988889,0.846911,NaN
8783,2021-01-01 23:00:00,1,728.23,731.97,725.65,728.90,728.916239,728.276445,729.266965,14614.203530,...,0.848841,0.903274,0.964718,1.011494,-0.047061,-208.653580,0.829757,0.932976,0.835221,NaN
8784,2021-01-02 00:00:00,0,29262.32,29338.89,29228.14,29331.69,29289.668860,29263.372930,29287.525770,971.123385,...,0.888754,0.916514,0.991179,1.065321,-0.204856,-724.018546,0.853797,0.988672,0.846911,NaN


### Feature Engineering

#### 因子挖掘框架

In [21]:
class Factor_Mining:
    def __init__(self, data) -> None:
        self.data = data
        
    def cal_rv(self, window=24):
        assert np.isin(['close'], data.columns).all()
        _min_periods = int(0.8 * window)
        _rv = np.sqrt(np.square(np.log(data['close']).diff()).rolling(window = int(window), min_periods = _min_periods).mean() * int(365*24))
        _rv =  _rv.shift(-24)
        return _rv
    
    def cal_ic(self, fac_symbol, space=None):
        # [(para_11, para_12)]
        # [para_1, para_2]
        rv_data = self.cal_rv()
        if isinstance(space, list):
            ic_collect = []
            for para in space:
                if isinstance(para, tuple):
                    fac_data = fac_symbol(self.data.copy(), *para)
                else:
                     fac_data = fac_symbol(self.data.copy(), para)
                res = pd.DataFrame({'fac':fac_data, 'rv':rv_data}).dropna()
                ic = res['fac'].corr(res['rv'])
                ic_collect.append(ic)
            ic_max = max(ic_collect, key=lambda x: abs(x))
            para_max = space[ic_collect.index(ic_max)]
            return ic_max, para_max
        else:
            fac_data = fac_symbol(self.data.copy())
            res = pd.DataFrame({'fac':fac_data, 'rv':rv_data}).dropna()
            ic = res['fac'].corr(res['rv'])
            return ic 
            
            
                
            
           

In [25]:
def corr_high_rank_volume(data:pd.DataFrame, w:int=168):
        assert np.isin(['high', 'volume'], data.columns).all()
        rank_vol = data['volume'].rank()
        _res = data['high'].rolling(window=w, min_periods=int(0.8*w)).corr(rank_vol)
        return _res
COL_NAMES = ['open', 'high', 'low', 'close', 'vwap', 'twap', 'awap', 'volume','tradeval', 'ntrade', 'ntrade_up', 'ntrade_down', 'ntrade_flat', 'close_ask', 'close_bid', 'close_asksize', 'close_bidsize', 'close_spread', 'nquote']
ROOT_DIR = '/Users/zehoo/Documents/Course/波动率预测专题课程/data/DC_Features_20190401_20221224.parquet'
dm_btc = DataModule(source=ROOT_DIR, freq='1h', base='BTC')
data = dm_btc.get_data(starttime='20200102', endtime='20210102', columns=COL_NAMES)
fm = Factor_Mining(data)
# no_space
# fm.cal_ic(corr_high_rank_volume)
# with space
fm.cal_ic(corr_high_rank_volume, space=[72, 120, 168])


/var/folders/rc/d92q64ds74d0jg428sqj64lc0000gn/T/ipykernel_70351/990086325.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_base = df_clip[df['base'] == base]


(-0.23343022543137043, 168)

In [26]:
import pandas as pd
import numpy as np

class Features:
    @staticmethod
    def price_norm(data:pd.DataFrame, column:str, windows) -> np.array:
        # 用于归一化价格量纲，表示相对价格大小
        assert np.isin([column], data.columns).all()
        _min_periods = (windows-1)*24
        return data[column]/data[column].rolling(windows*24, min_periods=_min_periods).mean()

    @staticmethod
    def log_return(data:pd.DataFrame) -> np.array:
        # 计算对数收益率
        assert np.isin(['close'], data.columns).all()
        _lr = np.log(data['close']).diff()
        return _lr.values

    @staticmethod
    def rv(data:pd.DataFrame, window:float, scale_window:float, shift:int=0) -> np.array:
        # 计算波动率
        assert np.isin(['close'], data.columns).all()
        _min_periods = int(0.8 * window) # rolling winsows中包含的最小的元素数量
        _rv = np.sqrt(np.square(np.log(data['close']).diff()).rolling(window = int(window), min_periods = _min_periods).mean() * int(scale_window)) # scale_window 365*24 处理到年化水平
        _rv =  _rv.shift(shift) # shift 预测的长度
        return _rv.values

    @staticmethod
    def rv_24(data:pd.DataFrame, window:float, scale_window:float) -> np.array:
        assert np.isin(['close'], data.columns).all()
        _rv_24 = Features.rv(data, window, scale_window, shift=-24)
        return _rv_24

    @staticmethod
    def BookSkew(data: pd.DataFrame) -> np.array:
        """
        计算盘口偏度（skew）指标，该指标用于分析市场的买卖压力分布。

        参数:
        data (pd.DataFrame): 包含金融数据的DataFrame，必须包含 'close_asksize' 和 'close_bidsize' 列。

        返回:
        np.array: 包含计算的盘口偏度值的数组。

        逻辑:
        1. 首先，使用断言确保 'data' 包含 'close_asksize' 和 'close_bidsize' 两列。
        2. 然后，计算盘口偏度（skew）指标，它用于描述卖单和买单的相对强度。
        3. 最后，返回包含盘口偏度值的数组。

        相关信息:
        - 盘口偏度是金融分析中的一个重要指标，用于衡量市场上买卖双方的力量对比。

        示例用法:
        data = pd.DataFrame(...)
        skew_values = YourClass.BookSkew(data)
        """
        assert np.isin(['close_asksize', 'close_bidsize'], data.columns).all()
        
        # 计算盘口偏度（skew）
        _skew = data.close_bidsize / (data.close_asksize + data.close_bidsize)
        
        return _skew.values



    @staticmethod
    def DayofWeek(data: pd.DataFrame):
        """
        计算DataFrame的日期索引对应的每个日期是一周的哪一天（星期几）。

        参数:
        data (pd.DataFrame): 包含日期索引的DataFrame。

        返回:
        np.array: 包含每个日期的星期几的数组。

        逻辑:
        1. 首先，将DataFrame的索引转换为日期时间格式，以便处理日期信息。
        2. 然后，使用Pandas的`weekday`属性获取每个日期的星期几（0表示星期一，1表示星期二，...，6表示星期日）。
        3. 最后，返回包含星期几信息的数组。

        示例用法:
        data = pd.DataFrame(...)
        dow = YourClass.DayofWeek(data)
        """
        index_ = pd.to_datetime(data.index.get_level_values(0))
        _dow = index_.weekday
        return _dow.values


    @staticmethod
    def TimeFromMidnight(data: pd.DataFrame):
        """
        计算DataFrame日期索引中每个时间点距午夜的时间（以小数天为单位）。

        参数:
        data (pd.DataFrame): 包含日期索引的DataFrame。

        返回:
        np.array: 包含每个时间点距午夜的时间的数组（以小数天为单位）。

        逻辑:
        1. 首先，将DataFrame的索引转换为日期时间格式，以便处理日期和时间信息。
        2. 然后，计算每个时间点距午夜的时间，将结果转换为小数天。
        3. 最后，返回包含时间信息的数组。

        示例用法:
        data = pd.DataFrame(...)
        time_since_midnight = YourClass.TimeFromMidnight(data)
        """
        index_ = pd.to_datetime(data.index.get_level_values(0))
        
        # 计算每个时间点距午夜的时间
        _time_since = (index_ - index_.normalize()).seconds / 86400
        
        return _time_since.values

    
    @staticmethod
    def HighOrLow(data: pd.DataFrame, period: int, is_max: bool = False):
        """
        计算DataFrame中市场价格相对于一定时间内最高价或最低价的涨幅或跌幅。

        参数:
        data (pd.DataFrame): 包含金融数据的DataFrame，必须包含 'close_ask' 和 'close_bid' 列。
        period (int): 用于计算最高价或最低价的时间窗口大小。
        is_max (bool, 可选): 如果为True，则计算相对于最高价的涨幅；如果为False（默认），则计算相对于最低价的跌幅。

        返回:
        np.array: 包含相对于最高价或最低价的涨幅或跌幅的数组。

        逻辑:
        1. 首先，确保 'data' 包含 'close_ask' 和 'close_bid' 两列。
        2. 计算市场价格的中间价（(close_ask + close_bid) / 2）。
        3. 根据 'is_max' 参数，选择计算最高价或最低价的滚动窗口值。
        4. 计算相对于最高价或最低价的涨幅或跌幅。
        5. 返回包含涨幅或跌幅信息的数组。

        示例用法:
        data = pd.DataFrame(...)
        # 计算相对于最高价的涨幅
        high_rise = YourClass.HighOrLow(data, period=10, is_max=True)
        # 计算相对于最低价的跌幅
        low_fall = YourClass.HighOrLow(data, period=10, is_max=False)
        """
        assert np.isin(['close_ask', 'close_bid'], data.columns).all()
        
        # 计算市场价格的中间价
        _mp = (data.close_ask + data.close_bid) / 2
        
        # 根据 'is_max' 参数，选择计算最高价或最低价的滚动窗口值
        if is_max:
            _val = _mp.rolling(period).max()
        else:
            _val = _mp.rolling(period).min() 
        
        # 计算相对于最高价或最低价的涨幅或跌幅
        result = ((_val - _mp) / _mp).values
        
        return result


    @staticmethod
    def TT(data: pd.DataFrame, period: int):
        """
        计算DataFrame中市场订单总量的滚动加和。

        参数:
        data (pd.DataFrame): 包含金融数据的DataFrame，必须包含 'close_asksize' 和 'close_bidsize' 列。
        period (int): 用于计算滚动加和的时间窗口大小。

        返回:
        np.array: 包含滚动加和后的市场订单总量的数组。

        逻辑:
        1. 首先，确保 'data' 包含 'close_asksize' 和 'close_bidsize' 两列。
        2. 计算市场订单总量，即 'close_bidsize' 和 'close_asksize' 的总和。
        3. 使用滚动窗口计算总订单量在给定时间窗口内的累积和。
        4. 返回包含累积和的数组。

        示例用法:
        data = pd.DataFrame(...)
        total_order_size = YourClass.TT(data, period=10)
        """
        
        # 确保 'data' 包含 'close_asksize' 和 'close_bidsize' 两列
        assert np.isin(['close_asksize', 'close_bidsize'], data.columns).all()
        
        # 计算市场订单总量
        _size = data.close_bidsize + data.close_asksize
        
        # 使用滚动窗口计算总订单量在给定时间窗口内的累积和
        result = _size.rolling(period).sum().values
    
        return result


    @staticmethod
    def TTNet(data: pd.DataFrame, period: int):
        """
        计算DataFrame中市场订单净量的滚动加和。

        参数:
        data (pd.DataFrame): 包含金融数据的DataFrame，必须包含 'close_asksize' 和 'close_bidsize' 列。
        period (int): 用于计算滚动加和的时间窗口大小。

        返回:
        np.array: 包含滚动加和后的市场订单净量的数组。

        逻辑:
        1. 首先，确保 'data' 包含 'close_asksize' 和 'close_bidsize' 两列。
        2. 计算市场订单净量，即 'close_bidsize' 减去 'close_asksize'。
        3. 使用滚动窗口计算订单净量在给定时间窗口内的累积和。
        4. 返回包含累积和的数组。

        示例用法:
        data = pd.DataFrame(...)
        net_order_size = YourClass.TTNet(data, period=10)
        """
        
        # 确保 'data' 包含 'close_asksize' 和 'close_bidsize' 两列
        assert np.isin(['close_asksize', 'close_bidsize'], data.columns).all()
        
        # 计算市场订单净量
        _res = data.close_bidsize - data.close_asksize
        
        # 使用滚动窗口计算订单净量在给定时间窗口内的累积和
        result = _res.rolling(period).sum().values
        
        return result


    @staticmethod
    def TTNetNum(data: pd.DataFrame, period: int):
        """
        计算DataFrame中市场订单净量的滚动加和，每个时间点的正负号代表买入（+1）或卖出（-1）。

        参数:
        data (pd.DataFrame): 包含金融数据的DataFrame，必须包含 'close_asksize' 和 'close_bidsize' 列。
        period (int): 用于计算滚动加和的时间窗口大小。

        返回:
        np.array: 包含滚动加和后的市场订单净量的数组，每个时间点的正负号代表买入（+1）或卖出（-1）。

        逻辑:
        1. 首先，确保 'data' 包含 'close_asksize' 和 'close_bidsize' 两列。
        2. 计算市场订单净量，即 'close_bidsize' 减去 'close_asksize'。
        3. 使用Lambda函数将每个净量的正负号表示为+1（买入）或-1（卖出）。
        4. 使用滚动窗口计算订单净量在给定时间窗口内的累积和，同时保留正负号信息。
        5. 返回包含累积和的数组，每个时间点的正负号代表买入（+1）或卖出（-1）。

        示例用法:
        data = pd.DataFrame(...)
        net_order_size = YourClass.TTNetNum(data, period=10)
        """
    
        # 确保 'data' 包含 'close_asksize' 和 'close_bidsize' 两列
        assert np.isin(['close_asksize', 'close_bidsize'], data.columns).all()
        
        # 计算市场订单净量
        _res = (data.close_bidsize - data.close_asksize).apply(lambda x: np.sign(x))
        
        # 使用滚动窗口计算订单净量在给定时间窗口内的累积和，同时保留正负号信息
        result = _res.rolling(period).sum().values
        
        return result


    @staticmethod
    def TTNetNumPct(data: pd.DataFrame, period: int):
        """
        计算DataFrame中市场订单净量的滚动加和，每个时间点的正负号代表买入（+1）或卖出（-1），并将正负号表示为百分比形式。

        参数:
        data (pd.DataFrame): 包含金融数据的DataFrame，必须包含 'close_asksize' 和 'close_bidsize' 列。
        period (int): 用于计算滚动加和的时间窗口大小。

        返回:
        np.array: 包含滚动加和后的市场订单净量的数组，每个时间点的正负号代表买入（1.0）或卖出（0.0）。

        逻辑:
        1. 首先，确保 'data' 包含 'close_asksize' 和 'close_bidsize' 两列。
        2. 计算市场订单净量，即 'close_bidsize' 减去 'close_asksize'。
        3. 使用Lambda函数将每个净量的正负号表示为0.5（卖出）或1.0（买入）。
        4. 使用滚动窗口计算订单净量在给定时间窗口内的累积和，并将正负号表示为0.0（卖出）或1.0（买入）。
        5. 返回包含累积和的数组，每个时间点的正负号代表买入（1.0）或卖出（0.0）。

        示例用法:
        data = pd.DataFrame(...)
        net_order_size = YourClass.TTNetNumPct(data, period=10)
        """
        
        # 确保 'data' 包含 'close_asksize' 和 'close_bidsize' 两列
        assert np.isin(['close_asksize', 'close_bidsize'], data.columns).all()
        
        # 计算市场订单净量
        _res = (data.close_bidsize - data.close_asksize).apply(lambda x: 0.5 * np.sign(x) + 0.5)
        
        # 使用滚动窗口计算订单净量在给定时间窗口内的累积和，并将正负号表示为0.0（卖出）或1.0（买入）
        result = _res.rolling(period).sum().values
        
        return result


    @staticmethod
    def TTNum(data: pd.DataFrame, period: int):
        """
        计算DataFrame中市场订单净量的滚动加和，每个时间点的正负号代表买入（+1）或卖出（-1），并将正负号表示为绝对值。

        参数:
        data (pd.DataFrame): 包含金融数据的DataFrame，必须包含 'close_asksize' 和 'close_bidsize' 列。
        period (int): 用于计算滚动加和的时间窗口大小。

        返回:
        np.array: 包含滚动加和后的市场订单净量的数组，每个时间点的正负号都为1。

        逻辑:
        1. 首先，确保 'data' 包含 'close_asksize' 和 'close_bidsize' 两列。
        2. 计算市场订单净量，即 'close_bidsize' 减去 'close_asksize'。
        3. 使用Lambda函数将每个净量的正负号表示为1，表示无论是买入还是卖出，都将其视为正数。
        4. 使用滚动窗口计算订单净量在给定时间窗口内的累积和，并将正负号保持为1。
        5. 返回包含累积和的数组，每个时间点的正负号都为1。

        示例用法:
        data = pd.DataFrame(...)
        net_order_size = YourClass.TTNum(data, period=10)
        """
    
        # 确保 'data' 包含 'close_asksize' 和 'close_bidsize' 两列
        assert np.isin(['close_asksize', 'close_bidsize'], data.columns).all()
        
        # 计算市场订单净量
        _res = (data.close_bidsize - data.close_asksize).apply(lambda x: np.abs(np.sign(x)))
        
        # 使用滚动窗口计算订单净量在给定时间窗口内的累积和，并将正负号保持为1
        result = _res.rolling(period).sum().values
        
        return result


    @staticmethod
    def TTToLevelRatio(data: pd.DataFrame):
        """
        计算DataFrame中市场订单净量（买入净量 - 卖出净量）在每个时间点相对于总净量的比率。

        参数:
        data (pd.DataFrame): 包含金融数据的DataFrame，必须包含 'close_asksize' 和 'close_bidsize' 列。

        返回:
        np.array: 包含市场订单净量在每个时间点相对于总净量的比率的数组。

        逻辑:
        1. 首先，确保 'data' 包含 'close_asksize' 和 'close_bidsize' 两列。
        2. 计算市场的买入净量和卖出净量，使用累积的方式计算，正数表示买入，负数表示卖出。
        3. 使用这两个累积值计算市场订单净量在每个时间点相对于总净量的比率。
        4. 返回包含比率信息的数组。

        示例用法:
        data = pd.DataFrame(...)
        order_ratio = YourClass.TTToLevelRatio(data)
        """
        
        # 确保 'data' 包含 'close_asksize' 和 'close_bidsize' 两列
        assert np.isin(['close_asksize', 'close_bidsize'], data.columns).all()
        
        # 计算市场的买入净量和卖出净量，使用累积的方式计算，正数表示买入，负数表示卖出
        _bids = np.cumsum(np.signbit(data.close_bidsize - data.close_asksize) * (data.close_bidsize - data.close_asksize))
        _asks = np.cumsum(np.signbit(data.close_asksize - data.close_bidsize) * (data.close_asksize - data.close_bidsize))
        
        # 使用这两个累积值计算市场订单净量在每个时间点相对于总净量的比率
        result = (_bids / (_asks + _bids)).values
        
        return result


    @staticmethod
    def VolumeChangeRatio(data: pd.DataFrame):
        """
        计算DataFrame中市场订单的成交量变化比率。

        参数:
        data (pd.DataFrame): 包含金融数据的DataFrame，必须包含 'close_asksize' 和 'close_bidsize' 列。

        返回:
        np.array: 包含市场订单的成交量变化比率的数组。

        逻辑:
        1. 首先，确保 'data' 包含 'close_asksize' 和 'close_bidsize' 两列。
        2. 根据 'close_bidsize' 是否大于 'close_asksize'，将数据分为买单和卖单。
        3. 计算买单和卖单的成交量变化，忽略微小的变化（小于1e-5）。
        4. 计算每个时间点的成交量变化比率，根据买单和卖单的不同来选择对应的成交量变化。
        5. 返回包含成交量变化比率的数组。

        示例用法:
        data = pd.DataFrame(...)
        volume_ratio = YourClass.VolumeChangeRatio(data)
        """
    
        # 确保 'data' 包含 'close_asksize' 和 'close_bidsize' 两列
        assert np.isin(['close_asksize', 'close_bidsize'], data.columns).all()
        
        # 根据 'close_bidsize' 是否大于 'close_asksize'，将数据分为买单和卖单
        is_bid = data.close_bidsize > data.close_asksize
        is_ask = data.close_bidsize <= data.close_asksize
        
        # 计算买单和卖单的成交量变化，忽略微小的变化（小于1e-5）
        _bidsize = ((data.close_bidsize.diff() > 1e-5) * data.close_bidsize +
                    (data.close_bidsize.diff() < -1e-5) * data.close_bidsize.shift(1) +
                    (data.close_bidsize.diff().abs() <= 1e-5) * data.close_bidsize.diff().abs())

        _asksize = ((data.close_asksize.diff() > 1e-5) * data.close_asksize +
                    (data.close_asksize.diff() < -1e-5) * data.close_asksize.shift(1) +
                    (data.close_asksize.diff().abs() <= 1e-5) * data.close_asksize.diff().abs())
        
        # 计算每个时间点的成交量变化比率，根据买单和卖单的不同来选择对应的成交量变化
        result = (is_bid * _bidsize + is_ask * _asksize).values
        
        return result

    
    @staticmethod
    def vwap_balance(data: pd.DataFrame, is_abs=True):
        """
        计算DataFrame中的VWAP（成交量加权平均价）买单和卖单之间的差值。

        参数:
        data (pd.DataFrame): 包含金融数据的DataFrame，必须包含 'vwap_buy' 和 'vwap_sell' 列。
        is_abs (bool, 可选): 控制是否返回差值的绝对值。默认为 True，返回绝对值。

        返回:
        np.array: 包含VWAP买单和卖单之间的差值或绝对差值的数组，根据 'is_abs' 参数而定。

        逻辑:
        1. 首先，确保 'data' 包含 'vwap_buy' 和 'vwap_sell' 两列。
        2. 计算VWAP买单和卖单之间的差值。
        3. 如果 'is_abs' 参数为 True，则返回差值的绝对值，否则返回原始差值。
        4. 返回包含差值或绝对差值的数组。

        示例用法:
        data = pd.DataFrame(...)
        vwap_diff = YourClass.vwap_balance(data)
        或
        vwap_abs_diff = YourClass.vwap_balance(data, is_abs=True)
        """
        
        # 确保 'data' 包含 'vwap_buy' 和 'vwap_sell' 两列
        assert np.isin(['vwap_buy', 'vwap_sell'], data.columns).all()
        
        # 计算VWAP买单和卖单之间的差值
        _res = data['vwap_buy'] - data['vwap_sell']
        
        # 如果 'is_abs' 参数为 True，则返回差值的绝对值，否则返回原始差值
        result = abs(_res) if is_abs else _res
        
        return result

    
    @staticmethod
    def volume_imbalance(data: pd.DataFrame, is_abs=True):
        """
        计算DataFrame中的买单和卖单成交量之间的差值。

        参数:
        data (pd.DataFrame): 包含金融数据的DataFrame，必须包含 'volume_buy' 和 'volume_sell' 列。
        is_abs (bool, 可选): 控制是否返回差值的绝对值。默认为 True，返回绝对值。

        返回:
        np.array: 包含买单和卖单成交量之间的差值或绝对差值的数组，根据 'is_abs' 参数而定。

        逻辑:
        1. 首先，确保 'data' 包含 'volume_buy' 和 'volume_sell' 两列。
        2. 计算买单成交量和卖单成交量之间的差值。
        3. 如果 'is_abs' 参数为 True，则返回差值的绝对值，否则返回原始差值。
        4. 返回包含差值或绝对差值的数组。

        示例用法:
        data = pd.DataFrame(...)
        volume_diff = YourClass.volume_imbalance(data)
        或
        volume_abs_diff = YourClass.volume_imbalance(data, is_abs=True)
        """
    
        # 确保 'data' 包含 'volume_buy' 和 'volume_sell' 两列
        assert np.isin(['volume_buy', 'volume_sell'], data.columns).all()
        
        # 计算买单成交量和卖单成交量之间的差值
        _res = data['volume_buy'] - data['volume_sell']
        
        # 如果 'is_abs' 参数为 True，则返回差值的绝对值，否则返回原始差值
        result = abs(_res) if is_abs else _res
        
        return result


    @staticmethod
    def tradeval_imbalance(data: pd.DataFrame, is_abs=True):
        """
        计算DataFrame中的买单和卖单成交额之间的差值。

        参数:
        data (pd.DataFrame): 包含金融数据的DataFrame，必须包含 'tradeval_buy' 和 'tradeval_sell' 列。
        is_abs (bool, 可选): 控制是否返回差值的绝对值。默认为 True，返回绝对值。

        返回:
        np.array: 包含买单和卖单成交额之间的差值或绝对差值的数组，根据 'is_abs' 参数而定。

        逻辑:
        1. 首先，确保 'data' 包含 'tradeval_buy' 和 'tradeval_sell' 两列。
        2. 计算买单成交成交额和卖单成交额值之间的差值。
        3. 如果 'is_abs' 参数为 True，则返回差值的绝对值，否则返回原始差值。
        4. 返回包含差值或绝对差值的数组。

        示例用法:
        data = pd.DataFrame(...)
        tradeval_diff = YourClass.tradeval_imbalance(data)
        或
        tradeval_abs_diff = YourClass.tradeval_imbalance(data, is_abs=True)
        """
    
        # 确保 'data' 包含 'tradeval_buy' 和 'tradeval_sell' 两列
        assert np.isin(['tradeval_buy', 'tradeval_sell'], data.columns).all()
        
        # 计算买单成交额和卖单成交额之间的差值
        _res = data['tradeval_buy'] - data['tradeval_sell']
        
        # 如果 'is_abs' 参数为 True，则返回差值的绝对值，否则返回原始差值
        result = abs(_res) if is_abs else _res
        
        return result


    @staticmethod
    def ntrade_imbalance(data: pd.DataFrame, is_abs=True):
        """
        计算DataFrame中的买单和卖单成交笔数之间的差值。

        参数:
        data (pd.DataFrame): 包含金融数据的DataFrame，必须包含 'ntrade_buy' 和 'ntrade_sell' 列。
        is_abs (bool, 可选): 控制是否返回差值的绝对值。默认为 True，返回绝对值。

        返回:
        np.array: 包含买单和卖单成交笔数之间的差值或绝对差值的数组，根据 'is_abs' 参数而定。

        逻辑:
        1. 首先，确保 'data' 包含 'ntrade_buy' 和 'ntrade_sell' 两列。
        2. 计算买单成交笔数和卖单成交笔数之间的差值。
        3. 如果 'is_abs' 参数为 True，则返回差值的绝对值，否则返回原始差值。
        4. 返回包含差值或绝对差值的数组。

        示例用法:
        data = pd.DataFrame(...)
        ntrade_diff = YourClass.ntrade_imbalance(data)
        或
        ntrade_abs_diff = YourClass.ntrade_imbalance(data, is_abs=True)
        """
    
        # 确保 'data' 包含 'ntrade_buy' 和 'ntrade_sell' 两列
        assert np.isin(['ntrade_buy', 'ntrade_sell'], data.columns).all()
        
        # 计算买单成交笔数和卖单成交笔数之间的差值
        _res = data['ntrade_buy'] - data['ntrade_sell']
        
        # 如果 'is_abs' 参数为 True，则返回差值的绝对值，否则返回原始差值
        result = abs(_res) if is_abs else _res
        
        return result


    @staticmethod
    def ntrade_up_imbalance(data: pd.DataFrame, is_abs=True):
        """
        计算DataFrame中的买单和卖单成交上涨笔数之间的差值。

        参数:
        data (pd.DataFrame): 包含金融数据的DataFrame，必须包含 'ntrade_up_buy' 和 'ntrade_up_sell' 列。
        is_abs (bool, 可选): 控制是否返回差值的绝对值。默认为 True，返回绝对值。

        返回:
        np.array: 包含买单和卖单成交上涨笔数之间的差值或绝对差值的数组，根据 'is_abs' 参数而定。

        逻辑:
        1. 首先，确保 'data' 包含 'ntrade_up_buy' 和 'ntrade_up_sell' 两列。
        2. 计算买单成交上涨笔数和卖单成交上涨笔数之间的差值。
        3. 如果 'is_abs' 参数为 True，则返回差值的绝对值，否则返回原始差值。
        4. 返回包含差值或绝对差值的数组。

        示例用法:
        data = pd.DataFrame(...)
        ntrade_up_diff = YourClass.ntrade_up_imbalance(data)
        或
        ntrade_up_abs_diff = YourClass.ntrade_up_imbalance(data, is_abs=True)
        """
        
        # 确保 'data' 包含 'ntrade_up_buy' 和 'ntrade_up_sell' 两列
        assert np.isin(['ntrade_up_buy', 'ntrade_up_sell'], data.columns).all()
        
        # 计算买单成交上涨笔数和卖单成交上涨笔数之间的差值
        _res = data['ntrade_up_buy'] - data['ntrade_up_sell']
        
        # 如果 'is_abs' 参数为 True，则返回差值的绝对值，否则返回原始差值
        result = abs(_res) if is_abs else _res
        
        return result


    @staticmethod
    def ntrade_down_imbalance(data: pd.DataFrame, is_abs=True):
        """
        计算DataFrame中的买单和卖单成交下跌笔数之间的差值。

        参数:
        data (pd.DataFrame): 包含金融数据的DataFrame，必须包含 'ntrade_down_buy' 和 'ntrade_down_sell' 列。
        is_abs (bool, 可选): 控制是否返回差值的绝对值。默认为 True，返回绝对值。

        返回:
        np.array: 包含买单和卖单成交下跌笔数之间的差值或绝对差值的数组，根据 'is_abs' 参数而定。

        逻辑:
        1. 首先，确保 'data' 包含 'ntrade_down_buy' 和 'ntrade_down_sell' 两列。
        2. 计算买单成交下跌笔数和卖单成交下跌笔数之间的差值。
        3. 如果 'is_abs' 参数为 True，则返回差值的绝对值，否则返回原始差值。
        4. 返回包含差值或绝对差值的数组。

        示例用法:
        data = pd.DataFrame(...)
        ntrade_down_diff = YourClass.ntrade_down_imbalance(data)
        或
        ntrade_down_abs_diff = YourClass.ntrade_down_imbalance(data, is_abs=True)
        """
    
        # 确保 'data' 包含 'ntrade_down_buy' 和 'ntrade_down_sell' 两列
        assert np.isin(['ntrade_down_buy', 'ntrade_down_sell'], data.columns).all()
        
        # 计算买单成交下跌笔数和卖单成交下跌笔数之间的差值
        _res = data['ntrade_down_buy'] - data['ntrade_down_sell']
        
        # 如果 'is_abs' 参数为 True，则返回差值的绝对值，否则返回原始差值
        result = abs(_res) if is_abs else _res
        
        return result


    @staticmethod
    def ntrade_flat_imbalance(data: pd.DataFrame, is_abs=True):
        """
        计算DataFrame中的买单和卖单成交平盘笔数之间的差值。

        参数:
        data (pd.DataFrame): 包含金融数据的DataFrame，必须包含 'ntrade_flat_buy' 和 'ntrade_flat_sell' 列。
        is_abs (bool, 可选): 控制是否返回差值的绝对值。默认为 True，返回绝对值。

        返回:
        np.array: 包含买单和卖单成交平盘笔数之间的差值或绝对差值的数组，根据 'is_abs' 参数而定。

        逻辑:
        1. 首先，确保 'data' 包含 'ntrade_flat_buy' 和 'ntrade_flat_sell' 两列。
        2. 计算买单成交平盘笔数和卖单成交平盘笔数之间的差值。
        3. 如果 'is_abs' 参数为 True，则返回差值的绝对值，否则返回原始差值。
        4. 返回包含差值或绝对差值的数组。

        示例用法:
        data = pd.DataFrame(...)
        ntrade_flat_diff = YourClass.ntrade_flat_imbalance(data)
        或
        ntrade_flat_abs_diff = YourClass.ntrade_flat_imbalance(data, is_abs=True)
        """
    
        # 确保 'data' 包含 'ntrade_flat_buy' 和 'ntrade_flat_sell' 两列
        assert np.isin(['ntrade_flat_buy', 'ntrade_flat_sell'], data.columns).all()
        
        # 计算买单成交平盘笔数和卖单成交平盘笔数之间的差值
        _res = data['ntrade_flat_buy'] - data['ntrade_flat_sell']
        
        # 如果 'is_abs' 参数为 True，则返回差值的绝对值，否则返回原始差值
        result = abs(_res) if is_abs else _res
        
        return result

    
 
    @staticmethod
    def relative_trade_volume(data: pd.DataFrame, sw: int = 24, lw: int = 720):
        """
        计算DataFrame中的相对交易量。

        参数:
        data (pd.DataFrame): 包含金融数据的DataFrame，必须包含 'volume' 列。
        sw (int, 可选): 短期窗口大小，默认为 24。
        lw (int, 可选): 长期窗口大小，默认为 720。

        返回:
        np.array: 包含相对交易量的数组。

        逻辑:
        1. 首先，确保 'data' 包含 'volume' 列。
        2. 使用滚动窗口计算短期窗口内的平均交易量和长期窗口内的平均交易量。
        3. 计算短期平均交易量与长期平均交易量的比值，得到相对交易量。
        4. 返回包含相对交易量的数组。

        示例用法:
        data = pd.DataFrame(...)
        relative_volume = YourClass.relative_trade_volume(data, sw=24, lw=720)
        """
    
        # 确保 'data' 包含 'volume' 列
        assert np.isin(['volume'], data.columns)
        
        # 使用滚动窗口计算短期窗口内的平均交易量和长期窗口内的平均交易量
        short_term_avg = data['volume'].rolling(window=sw, min_periods=int(0.8 * sw)).mean()
        long_term_avg = data['volume'].rolling(window=lw, min_periods=int(0.8 * lw)).mean()
        
        # 计算短期平均交易量与长期平均交易量的比值，得到相对交易量
        _res = short_term_avg / long_term_avg
        
        return _res

    
    @staticmethod
    def relative_tradeval(data: pd.DataFrame, sw: int = 24, lw: int = 720):
        """
        计算DataFrame中的相对交易价值。

        参数:
        data (pd.DataFrame): 包含金融数据的DataFrame，必须包含 'tradeval' 列。
        sw (int, 可选): 短期窗口大小，默认为 24。
        lw (int, 可选): 长期窗口大小，默认为 720。

        返回:
        np.array: 包含相对交易价值的数组。

        逻辑:
        1. 首先，确保 'data' 包含 'tradeval' 列。
        2. 使用滚动窗口计算短期窗口内的平均交易价值和长期窗口内的平均交易价值。
        3. 计算短期平均交易价值与长期平均交易价值的比值，得到相对交易价值。
        4. 返回包含相对交易价值的数组。

        示例用法:
        data = pd.DataFrame(...)
        relative_tradeval = YourClass.relative_tradeval(data, sw=24, lw=720)
        """
    
        # 确保 'data' 包含 'tradeval' 列
        assert np.isin(['tradeval'], data.columns)
        
        # 使用滚动窗口计算短期窗口内的平均交易价值和长期窗口内的平均交易价值
        short_term_avg = data['tradeval'].rolling(window=sw, min_periods=int(0.8 * sw)).mean()
        long_term_avg = data['tradeval'].rolling(window=lw, min_periods=int(0.8 * lw)).mean()
        
        # 计算短期平均交易价值与长期平均交易价值的比值，得到相对交易价值
        _res = short_term_avg / long_term_avg
        
        return _res


    @staticmethod
    def relative_ntrade(data: pd.DataFrame, sw: int = 24, lw: int = 720):
        """
        计算DataFrame中的相对成交笔数。

        参数:
        data (pd.DataFrame): 包含金融数据的DataFrame，必须包含 'ntrade' 列。
        sw (int, 可选): 短期窗口大小，默认为 24。
        lw (int, 可选): 长期窗口大小，默认为 720。

        返回:
        np.array: 包含相对成交笔数的数组。

        逻辑:
        1. 首先，确保 'data' 包含 'ntrade' 列。
        2. 使用滚动窗口计算短期窗口内的平均成交笔数和长期窗口内的平均成交笔数。
        3. 计算短期平均成交笔数与长期平均成交笔数的比值，得到相对成交笔数。
        4. 返回包含相对成交笔数的数组。

        示例用法:
        data = pd.DataFrame(...)
        relative_ntrade = YourClass.relative_ntrade(data, sw=24, lw=720)
        """
    
        # 确保 'data' 包含 'ntrade' 列
        assert np.isin(['ntrade'], data.columns)
        
        # 使用滚动窗口计算短期窗口内的平均成交笔数和长期窗口内的平均成交笔数
        short_term_avg = data['ntrade'].rolling(window=sw, min_periods=int(0.8 * sw)).mean()
        long_term_avg = data['ntrade'].rolling(window=lw, min_periods=int(0.8 * lw)).mean()
        
        # 计算短期平均成交笔数与长期平均成交笔数的比值，得到相对成交笔数
        _res = short_term_avg / long_term_avg
        
        return _res

    
    @staticmethod
    def relative_high_ratio(data: pd.DataFrame, sw: int = 3, lw: int = 72):
        """
        计算DataFrame中的相对最高价比率。

        参数:
        data (pd.DataFrame): 包含金融数据的DataFrame，必须包含 'high' 列。
        sw (int, 可选): 短期窗口大小，默认为 3。
        lw (int, 可选): 长期窗口大小，默认为 72。

        返回:
        np.array: 包含相对最高价比率的数组。

        逻辑:
        1. 首先，确保 'data' 包含 'high' 列。
        2. 使用滚动窗口计算短期窗口内的最高价和长期窗口内的最高价。
        3. 计算短期最高价与长期最高价的比值，得到相对最高价比率。
        4. 返回包含相对最高价比率的数组。

        示例用法:
        data = pd.DataFrame(...)
        relative_high = YourClass.relative_high_ratio(data, sw=3, lw=72)
        """
    
        # 确保 'data' 包含 'high' 列
        assert np.isin(['high'], data.columns)
        
        # 使用滚动窗口计算短期窗口内的最高价和长期窗口内的最高价
        short_term_max = data['high'].rolling(window=sw, min_periods=int(0.8 * sw)).max()
        long_term_max = data['high'].rolling(window=lw, min_periods=int(0.8 * lw)).max()
        
        # 计算短期最高价与长期最高价的比值，得到相对最高价比率
        _res = short_term_max / long_term_max
        
        return _res


    @staticmethod
    def relative_low_ratio(data: pd.DataFrame, sw: int = 3, lw: int = 72):
        """
        计算DataFrame中的相对最低价比率。

        参数:
        data (pd.DataFrame): 包含金融数据的DataFrame，必须包含 'low' 列。
        sw (int, 可选): 短期窗口大小，默认为 3。
        lw (int, 可选): 长期窗口大小，默认为 72。

        返回:
        np.array: 包含相对最低价比率的数组。

        逻辑:
        1. 首先，确保 'data' 包含 'low' 列。
        2. 使用滚动窗口计算短期窗口内的最低价和长期窗口内的最低价。
        3. 计算短期最低价与长期最低价的比值，得到相对最低价比率。
        4. 返回包含相对最低价比率的数组。

        示例用法:
        data = pd.DataFrame(...)
        relative_low = YourClass.relative_low_ratio(data, sw=3, lw=72)
        """
    
        # 确保 'data' 包含 'low' 列
        assert np.isin(['low'], data.columns)
        
        # 使用滚动窗口计算短期窗口内的最低价和长期窗口内的最低价
        short_term_min = data['low'].rolling(window=sw, min_periods=int(0.8 * sw)).min()
        long_term_min = data['low'].rolling(window=lw, min_periods=int(0.8 * lw)).min()
        
        # 计算短期最低价与长期最低价的比值，得到相对最低价比率
        _res = short_term_min / long_term_min
        
        return _res


    @staticmethod
    def relative_spread(data: pd.DataFrame, sw: int = 72, lw: int = 720):
        """
        计算DataFrame中的相对价差比率。

        参数:
        data (pd.DataFrame): 包含金融数据的DataFrame，必须包含 'close_ask' 和 'close_bid' 列。
        sw (int, 可选): 短期窗口大小，默认为 72。
        lw (int, 可选): 长期窗口大小，默认为 720。

        返回:
        np.array: 包含相对价差比率的数组。

        逻辑:
        1. 首先，确保 'data' 包含 'close_ask' 和 'close_bid' 列。
        2. 计算价差 (spread) = 'close_ask' - 'close_bid'。
        3. 使用滚动窗口计算短期窗口内的价差平均值和长期窗口内的价差平均值。
        4. 计算短期价差平均值与长期价差平均值的比值，得到相对价差比率。
        5. 返回包含相对价差比率的数组。

        示例用法:
        data = pd.DataFrame(...)
        relative_spread = YourClass.relative_spread(data, sw=72, lw=720)
        """
    
        # 确保 'data' 包含 'close_ask' 和 'close_bid' 列
        assert np.isin(['close_ask', 'close_bid'], data.columns).all()
        
        # 计算价差 (spread) = 'close_ask' - 'close_bid'
        data['spread'] = data['close_ask'] - data['close_bid']
        
        # 使用滚动窗口计算短期窗口内的价差平均值和长期窗口内的价差平均值
        short_term_avg = data['spread'].rolling(window=sw, min_periods=int(0.8 * sw)).mean()
        long_term_avg = data['spread'].rolling(window=lw, min_periods=int(0.8 * lw)).mean()
        
        # 计算短期价差平均值与长期价差平均值的比值，得到相对价差比率
        _res = short_term_avg / long_term_avg
        
        return _res

  
    @staticmethod
    def corr_high_low(data: pd.DataFrame, w: int = 120):
        """
        计算DataFrame中最高价和最低价的滚动相关系数。

        参数:
        data (pd.DataFrame): 包含金融数据的DataFrame，必须包含 'high' 和 'low' 列。
        w (int, 可选): 滚动窗口大小，默认为 120。

        返回:
        np.array: 包含最高价和最低价的滚动相关系数的数组。

        逻辑:
        1. 首先，确保 'data' 包含 'high' 和 'low' 列。
        2. 使用滚动窗口计算最高价和最低价之间的滚动相关系数。
        3. 返回包含滚动相关系数的数组。

        示例用法:
        data = pd.DataFrame(...)
        corr_high_low = YourClass.corr_high_low(data, w=120)
        """
        
        # 确保 'data' 包含 'high' 和 'low' 列
        assert np.isin(['high', 'low'], data.columns).all()
        
        # 使用滚动窗口计算最高价和最低价之间的滚动相关系数
        _res = data['high'].rolling(window=w, min_periods=int(0.8 * w)).corr(data['low'])
        
        return _res


    @staticmethod
    def super_high_low(data: pd.DataFrame, w: int = 168):
        """
        计算DataFrame中的超级最高价与最低价比率。

        参数:
        data (pd.DataFrame): 包含金融数据的DataFrame，必须包含 'high' 和 'low' 列。
        w (int, 可选): 滚动窗口大小，默认为 168。

        返回:
        np.array: 包含最高价与最低价比率的数组。

        逻辑:
        1. 首先，确保 'data' 包含 'high' 和 'low' 列。
        2. 使用滚动窗口计算最高价和最低价的滚动最小值和滚动最大值。
        3. 计算滚动最小值与滚动最大值的比率，得到超级最高价与最低价比率。
        4. 返回包含超级最高价与最低价比率的数组。

        示例用法:
        data = pd.DataFrame(...)
        super_high_low = YourClass.super_high_low(data, w=168)
        """
        
        # 确保 'data' 包含 'high' 和 'low' 列
        assert np.isin(['high', 'low'], data.columns).all()
        
        # 使用滚动窗口计算最高价和最低价的滚动最小值和滚动最大值
        rolling_min_high = data['high'].rolling(window=w, min_periods=int(0.8 * w)).min()
        rolling_max_low = data['low'].rolling(window=w, min_periods=int(0.8 * w)).max()
    
        # 计算滚动最小值与滚动最大值的比率，得到最高价与最低价比率
        _res = rolling_min_high / rolling_max_low
        
        return _res


    @staticmethod
    def corr_high_rank_volume(data: pd.DataFrame, w: int = 168):
        """
        计算DataFrame中最高价和成交量的滚动相关系数。

        参数:
        data (pd.DataFrame): 包含金融数据的DataFrame，必须包含 'high' 和 'volume' 列。
        w (int, 可选): 滚动窗口大小，默认为 168。

        返回:
        np.array: 包含最高价和成交量的滚动相关系数的数组。

        逻辑:
        1. 首先，确保 'data' 包含 'high' 和 'volume' 列。
        2. 对成交量列进行排名操作，得到排名后的成交量。
        3. 使用滚动窗口计算最高价和排名后的成交量之间的滚动相关系数。
        4. 返回包含滚动相关系数的数组。

        示例用法:
        data = pd DataFrame(...)
        corr_high_rank_volume = YourClass.corr_high_rank_volume(data, w=168)
        """
    
        # 确保 'data' 包含 'high' 和 'volume' 列
        assert np.isin(['high', 'volume'], data.columns).all()
        
        # 对成交量列进行排名操作，得到排名后的成交量
        rank_vol = data['volume'].rank()
        
        # 使用滚动窗口计算最高价和排名后的成交量之间的滚动相关系数
        _res = data['high'].rolling(window=w, min_periods=int(0.8 * w)).corr(rank_vol)
        
        return _res



    @staticmethod
    def price_diff_ratio(data: pd.DataFrame, sw: int = 24, lw: int = 720):
        """
        计算DataFrame中的价格差异比率。

        参数:
        data (pd.DataFrame): 包含金融数据的DataFrame，必须包含 'vwap' 列。
        sw (int, 可选): 短期窗口大小，默认为 24。
        lw (int, 可选): 长期窗口大小，默认为 720。

        返回:
        np.array: 包含价格差异比率的数组。

        逻辑:
        1. 首先，确保 'data' 包含 'vwap' 列。
        2. 计算 VWAP（成交量加权平均价格）差异的绝对值，即每个时刻的 VWAP 与前一个时刻的 VWAP 差值的绝对值。
        3. 使用滚动窗口计算短期窗口内的 VWAP 差异平均值和长期窗口内的 VWAP 差异平均值。
        4. 计算短期 VWAP 差异平均值与长期 VWAP 差异平均值的比值，得到价格差异比率。
        5. 返回包含价格差异比率的数组。

        示例用法:
        data = pd.DataFrame(...)
        price_diff_ratio = YourClass.price_diff_ratio(data, sw=24, lw=720)
        """
    
        # 确保 'data' 包含 'vwap' 列
        assert np.isin(['vwap'], data.columns).all()
        
        # 计算 VWAP 差异的绝对值
        vwap_diff = data['vwap'].diff().abs()
        
        # 使用滚动窗口计算短期窗口内的 VWAP 差异平均值和长期窗口内的 VWAP 差异平均值
        short_term_avg = vwap_diff.rolling(window=sw, min_periods=int(0.8 * sw)).mean()
        long_term_avg = vwap_diff.rolling(window=lw, min_periods=int(0.8 * lw)).mean()
        
        # 计算短期 VWAP 差异平均值与长期 VWAP 差异平均值的比值，得到价格差异比率
        _res = short_term_avg / long_term_avg
        
        return _res



    @staticmethod
    def corr_std_high_vol(data: pd.DataFrame, w: int = 168):
        """
        计算DataFrame中最高价和成交量的滚动相关系数与滚动标准差乘积。

        参数:
        data (pd.DataFrame): 包含金融数据的DataFrame，必须包含 'high' 和 'volume' 列。
        w (int, 可选): 滚动窗口大小，默认为 168。

        返回:
        np.array: 包含最高价和成交量的滚动相关系数与滚动标准差乘积的数组。

        逻辑:
        1. 首先，确保 'data' 包含 'high' 和 'volume' 列。
        2. 使用滚动窗口计算最高价和成交量之间的滚动相关系数。
        3. 使用滚动窗口计算最高价的滚动标准差。
        4. 将滚动相关系数与滚动标准差排名的乘积作为结果。
        5. 返回包含结果的数组。

        示例用法:
        data = pd.DataFrame(...)
        corr_std_high_vol = YourClass.corr_std_high_vol(data, w=168)
        """
    
        # 确保 'data' 包含 'high' 和 'volume' 列
        assert np.isin(['high', 'volume'], data.columns).all()
        
        # 使用滚动窗口计算最高价和成交量之间的滚动相关系数
        corr_high_vol = data['high'].rolling(window=w, min_periods=int(0.8 * w)).corr(data['volume'])
        
        # 使用滚动窗口计算最高价的滚动标准差
        std_high = data['high'].rolling(window=w, min_periods=int(0.8 * w)).std()
        
        # 将滚动相关系数与滚动标准差排名的乘积作为结果
        _res = corr_high_vol * std_high.rank()
        
        return _res



 

### Expanding DataModule

In [35]:
class DataModule:
    def __init__(self, source, freq:str, base:str='BTC') -> None:
        self.freq = freq
        self.base = base
        self.source = source
    
    def get_data(self, starttime, endtime, columns):
        df = load_data(start_date=starttime, end_date=endtime, base=self.base.lower(), path=self.source)
        df = df.sort_values(by=['date','base'], ascending=True)
        df.set_index(['date', 'base'], drop=True, inplace=True)
        df = df[columns]
        return df
    
    def get_features(self, starttime, endtime, columns, add_target=True):
        feat = Features()
        df = self.get_data(starttime, endtime, columns)
        df['BookSkew'] = feat.BookSkew(df)
        df['DayofWeek'] = feat.DayofWeek(df)
        df['TimeFromNight'] = feat.TimeFromMidnight(df)
        df['HighOrLow_24'] = feat.HighOrLow(df, period=24)
        df['HighOrLow_3'] = feat.HighOrLow(df, period=3)
        df['HighOrLow_24_max'] = feat.HighOrLow(df, period=24, is_max=True)
        df['HighOrLow_3_max'] = feat.HighOrLow(df, period=3, is_max=True)
        df['relative_spread'] = feat.relative_spread(df)
        df['TT_24'] = feat.TT(df, period=24)
        df['TT_3'] = feat.TT(df, period=3)
        df['TTNet_24'] = feat.TTNet(df, period=24)
        df['TTNet_3'] = feat.TTNet(df, period=3)
        df['TTNetNum_24'] = feat.TTNetNum(df, period=24)
        df['TTNetNum_3'] = feat.TTNetNum(df, period=3)
        df['TTNetNumPct_24'] = feat.TTNetNumPct(df, period=24)
        df['TTNetNumPct_3'] = feat.TTNetNumPct(df, period=3)
        df['TTNum_24'] = feat.TTNum(df, period=24)
        df['TTNum_3'] = feat.TTNum(df, period=3)
        df['TTToLevelRatio'] = feat.TTToLevelRatio(df)
        df['VolumeChangeRatio'] = feat.VolumeChangeRatio(df)
        df['open_norm'] = feat.price_norm(df, column='open', windows=7)
        df['close_norm'] = feat.price_norm(df, column='close', windows=7)
        df['low_norm'] = feat.price_norm(df, column='low', windows=7)
        df['high_norm'] = feat.price_norm(df, column='high', windows=7)
        df['vwap_norm'] = feat.price_norm(df, column='vwap', windows=7)
        df['twap_norm'] = feat.price_norm(df, column='twap', windows=7)
        df['awap_norm'] = feat.price_norm(df, column='awap', windows=7)
        df['log_return'] = feat.log_return(df)
        df['rv_24before'] = feat.rv(df, window=24, scale_window=365 * 24)
        df["rv_1wbefore"] = feat.rv(df, window=24 * 7, scale_window=365 * 24)
        df["rv_1mbefore"] = feat.rv(df, window=24 * 30, scale_window=365 * 24)
        df['relative_trade_volume'] = feat.relative_trade_volume(df)
        df['relative_tradeval'] = feat.relative_tradeval(df)
        df['relative_ntrade'] = feat.relative_ntrade(df)
        df['relative_high_ratio'] = feat.relative_high_ratio(df)
        df['relative_low_ratio'] = feat.relative_low_ratio(df)
        df['corr_high_rank_volume'] = feat.corr_high_rank_volume(df)
        df['corr_std_high_vol'] = feat.corr_std_high_vol(df)
        df['price_diff_ratio'] = feat.price_diff_ratio(df)
        df['corr_high_low'] = feat.corr_high_low(df)
        df['super_high_low'] = feat.super_high_low(df)
        if add_target:
            df['rv_24'] = feat.rv_24(df, window=24, scale_window=365 * 24)
        return df

In [28]:
freq = '1h'
dm = [DataModule(ROOT_DIR, freq, base) for base in ['BTC', 'ETH']]
datasets = dict(zip(['BTC', 'ETH'], [dm_i.get_features(starttime='20200102', endtime='20210102', columns=COL_NAMES) for dm_i in dm]))
dp = DataPreprocessing(datasets=datasets, freq=freq)
data = dp.data_collation()

/var/folders/rc/d92q64ds74d0jg428sqj64lc0000gn/T/ipykernel_70351/990086325.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_base = df_clip[df['base'] == base]
/var/folders/rc/d92q64ds74d0jg428sqj64lc0000gn/T/ipykernel_70351/990086325.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_base = df_clip[df['base'] == base]
/var/folders/rc/d92q64ds74d0jg428sqj64lc0000gn/T/ipykernel_70351/3983087579.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_copy[column + '_lower_bound'] = df_copy[column + '_mean'] - tr * df_copy[column + '_std']
/var/folders/rc/d92q64ds74d0jg428sqj64lc0000gn/T/ipykernel_70351/3983087579.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usuall

In [30]:
data

,date,base,open,high,low,close,vwap,twap,awap,volume,...,relative_tradeval,relative_ntrade,relative_high_ratio,relative_low_ratio,corr_high_rank_volume,corr_std_high_vol,price_diff_ratio,corr_high_low,super_high_low,rv_24
0,2020-01-02 00:00:00,0,7197.62,7204.17,7175.15,7200.85,7188.047204,7197.715114,7189.798125,811.250160,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.402224
0,2020-01-02 00:00:00,1,130.28,130.87,129.74,130.77,130.270294,130.271626,130.293196,3865.459910,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.405712
1,2020-01-02 01:00:00,0,7200.08,7212.50,7185.46,7211.02,7200.446714,7200.821054,7200.155280,390.692131,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.409003
1,2020-01-02 01:00:00,1,130.71,130.78,130.27,130.67,130.551206,130.758253,130.524085,3769.696330,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.405849
2,2020-01-02 02:00:00,0,7211.04,7211.16,7185.11,7190.99,7194.751355,7195.567041,7195.165802,402.660139,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.426810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8782,2021-01-01 22:00:00,1,729.48,730.61,726.69,728.23,728.774104,729.453325,728.807794,14582.153160,...,0.844815,0.898394,0.963057,1.009324,-0.006285,-73.114013,0.839299,0.939944,0.833681,NaN
8783,2021-01-01 23:00:00,0,29163.17,29326.74,29104.57,29262.32,29255.752750,29164.501190,29253.241230,1127.035897,...,0.904756,0.928417,0.990768,1.071804,-0.194326,-664.458246,0.881032,0.988889,0.846911,NaN
8783,2021-01-01 23:00:00,1,728.23,731.97,725.65,728.90,728.916239,728.276445,729.266965,14614.203530,...,0.848841,0.903274,0.964718,1.011494,-0.047061,-208.653580,0.829757,0.932976,0.835221,NaN
8784,2021-01-02 00:00:00,0,29262.32,29338.89,29228.14,29331.69,29289.668860,29263.372930,29287.525770,971.123385,...,0.888754,0.916514,0.991179,1.065321,-0.204856,-724.018546,0.853797,0.988672,0.846911,NaN


In [1]:
import pandas as pd
df = pd.read_parquet('/Users/zehoo/Documents/Course/波动率预测专题课程/data/DC_Features_20190401_20221224.parquet')

In [29]:
df

,date,base,quote_x,open,high,low,close,volume,tradeval,ntrade,...,vwap_ask,twap_ask,awap_ask,open_asksize,high_asksize,low_asksize,close_asksize,vwa_asksize,twa_asksize,awa_asksize
13200,2020-01-01 00:00:00,btc,usdt,7200.52,7206.29,7185.76,7195.23,521.978397,3.755764e+06,6659,...,7195.993717,7195.455386,7195.633225,0.163159,8.656877,0.000001,0.031492,0.200801,0.175632,0.200791
13202,2020-01-01 01:00:00,btc,usdt,7195.24,7196.24,7175.46,7177.02,510.733521,3.668075e+06,7595,...,7182.098572,NaN,7181.940635,0.103091,3.899774,0.000001,0.228223,0.441340,NaN,0.441330
13204,2020-01-01 02:00:00,btc,usdt,7176.47,7230.00,7175.71,7216.27,883.052603,6.365953e+06,9033,...,7207.230252,7189.937233,7204.044510,0.228223,98.902845,0.000001,0.157927,0.787799,0.514536,0.787450
13206,2020-01-01 03:00:00,btc,usdt,7215.52,7244.87,7211.41,7242.85,655.156809,4.736719e+06,7466,...,7228.278972,7220.228511,7227.600090,0.008019,21.545003,0.000001,0.300000,0.576947,0.303303,0.576893
13208,2020-01-01 04:00:00,btc,usdt,7242.66,7245.00,7220.00,7225.01,783.724867,5.667367e+06,8337,...,7229.445759,7240.064157,7231.003755,0.640447,19.813351,0.000001,0.399879,0.669936,0.386292,0.670081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30760,2020-12-31 20:00:00,btc,usdt,28872.24,29000.00,28742.41,28897.83,2293.821339,6.628923e+07,56789,...,28899.936410,28896.987530,28896.765130,0.703973,52.079136,0.000001,0.244611,1.172446,1.151635,1.172317
30762,2020-12-31 21:00:00,btc,usdt,28897.84,29139.65,28862.00,29126.70,1936.480299,5.610330e+07,42101,...,28959.282340,28939.844050,28945.560650,0.243751,84.659713,0.000001,0.004500,1.164264,1.140160,1.163713
30764,2020-12-31 22:00:00,btc,usdt,29126.71,29169.55,28900.79,28966.36,2524.473111,7.335146e+07,51297,...,29061.244830,29049.671710,29051.604420,2.425720,38.106930,0.000001,0.491386,1.169931,1.140611,1.169543
30766,2020-12-31 23:00:00,btc,usdt,28966.36,29143.73,28910.19,29100.84,1438.506326,4.180712e+07,36413,...,29074.951920,29071.304470,29070.739290,0.501386,14.906356,0.000001,0.334847,0.659741,0.680326,0.659645


In [14]:
data

,date,base,open,high,low,close,vwap,twap,awap,volume,...,ntrade,ntrade_up,ntrade_down,ntrade_flat,close_ask,close_bid,close_asksize,close_bidsize,close_spread,nquote
0,2020-01-02 00:00:00,0,7197.62,7204.17,7175.15,7200.85,7188.047204,7197.715114,7189.798125,811.250160,...,7632,2573,2788,2271,7201.00,7200.14,0.102140,0.131905,0.86,16656
0,2020-01-02 00:00:00,1,130.28,130.87,129.74,130.77,130.270294,130.271626,130.293196,3865.459910,...,3232,893,768,1571,130.75,130.70,0.584810,32.294370,0.05,6246
1,2020-01-02 01:00:00,0,7200.08,7212.50,7185.46,7211.02,7200.446714,7200.821054,7200.155280,390.692131,...,5460,2014,1806,1640,7211.04,7210.69,0.346698,0.074837,0.35,11705
1,2020-01-02 01:00:00,1,130.71,130.78,130.27,130.67,130.551206,130.758253,130.524085,3769.696330,...,2563,703,652,1208,130.66,130.65,0.691260,19.135090,0.01,6692
2,2020-01-02 02:00:00,0,7211.04,7211.16,7185.11,7190.99,7194.751355,7195.567041,7195.165802,402.660139,...,5929,1927,2202,1800,7191.59,7190.99,0.347642,0.194713,0.60,10129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8782,2021-01-01 22:00:00,1,729.48,730.61,726.69,728.23,728.774104,729.453325,728.807794,14582.153160,...,10537,3094,3107,4336,728.26,728.20,7.918000,49.751950,0.06,17546
8783,2021-01-01 23:00:00,0,29163.17,29326.74,29104.57,29262.32,29255.752750,29164.501190,29253.241230,1127.035897,...,37533,10994,11311,15228,29262.32,29262.31,1.252100,0.101630,0.01,20393
8783,2021-01-01 23:00:00,1,728.23,731.97,725.65,728.90,728.916239,728.276445,729.266965,14614.203530,...,12435,3411,3374,5650,728.94,728.89,1.024090,0.066180,0.05,13786
8784,2021-01-02 00:00:00,0,29262.32,29338.89,29228.14,29331.69,29289.668860,29263.372930,29287.525770,971.123385,...,28620,7112,7099,14409,29331.70,29331.69,1.964657,0.000991,0.01,20353
